In [1]:
import pandas as pd
from collections import Counter
import re

import warnings
warnings.filterwarnings(action="ignore")

## Cleaning

In [2]:
data = pd.read_csv("nasdaqlisted.txt", sep = "|")
nsdq_df = data.loc[:,["Symbol", "Security Name"]]

In [3]:
nsdq_df

,Symbol,Security Name
0,AACG,ATA Creativity Global - American Depositary Sh...
1,AACI,Armada Acquisition Corp. I - Common Stock
2,AACIU,Armada Acquisition Corp. I - Unit
3,AACIW,Armada Acquisition Corp. I - Warrant
4,AADI,"Aadi Bioscience, Inc. - Common Stock"
...,...,...
5486,ZXZZT,NASDAQ TEST STOCK
5487,ZY,Zymergen Inc. - Common Stock
5488,ZYNE,"Zynerba Pharmaceuticals, Inc. - Common Stock"
5489,ZYXI,"Zynex, Inc. - Common Stock"


### Duplicated Rows

In [4]:
# check the uniqueness of each column 
print("symbols: ", nsdq_df.iloc[:,0].is_unique)
print("security name: ", nsdq_df.iloc[:,1].is_unique)

symbols:  True
security name:  False


In [5]:
# check the duplicated securities 
Counter(nsdq_df.iloc[:,1]).most_common(6)

[('NASDAQ TEST STOCK', 4),
 ('American Electric Power Company, Inc. - Corporate Unit', 2),
 ('Harbor Custom Development, Inc. - Warrant', 2),
 ('Hycroft Mining Holding Corporation - Warrant', 2),
 ('ATA Creativity Global - American Depositary Shares, each representing two common shares',
  1),
 ('Armada Acquisition Corp. I - Common Stock', 1)]

In [6]:
# check the duplicated securities 
nsdq_df.loc[nsdq_df["Security Name"].duplicated(keep = False)]

,Symbol,Security Name
125,AEPPL,"American Electric Power Company, Inc. - Corpor..."
126,AEPPZ,"American Electric Power Company, Inc. - Corpor..."
2328,HCDIW,"Harbor Custom Development, Inc. - Warrant"
2329,HCDIZ,"Harbor Custom Development, Inc. - Warrant"
2485,HYMCW,Hycroft Mining Holding Corporation - Warrant
2486,HYMCZ,Hycroft Mining Holding Corporation - Warrant
5463,ZJZZT,NASDAQ TEST STOCK
5480,ZVZZT,NASDAQ TEST STOCK
5484,ZWZZT,NASDAQ TEST STOCK
5486,ZXZZT,NASDAQ TEST STOCK


* except for NASDAQ TEST STOCK, the duplicated stock items may have been dual-listed

In [7]:
# remove NASDAQ TEST STOCK from the dataframe
drop_idx = list(nsdq_df[nsdq_df["Security Name"].duplicated(keep = False)][6:].index)
nsdq_df.drop(drop_idx, axis=0, inplace=True)

### Null Values

In [8]:
if nsdq_df["Security Name"].isnull().sum() | nsdq_df["Symbol"].isnull().sum() > 0:
    print("Null in Security Name: ", nsdq_df["Security Name"].isnull().sum())
    print("Null in Security Name: ", nsdq_df["Symbol"].isnull().sum())
    nsdq_df = nsdq_df.dropna()

Null in Security Name:  1
Null in Security Name:  0


### Text Cleaning
* Remove the security type info in Security Name column
* lowercase

In [14]:
def text_cleaner(text):
    regex1 = re.compile("(i*\s-[a-z0-9\s]*)|(\s-[a-z0-9\s]*)")
    regex2 = re.compile("(,*\s(inc).*)|(,*\s(corp).*)|(\s(ltd).*)")
    text_1 = regex1.sub('', text.lower())
    cleaned_text = regex2.sub('', text_1)
    return cleaned_text

In [ ]:
nsdq_df["Security Name"] = nsdq_df["Security Name"].apply(lambda x:text_cleaner(x))

In [15]:
nsdq_df.to_csv("/Users/alex/Desktop/deep_stock/nsdq_df.csv", index=True, header = True)